##c.3 ) use the pure tensorflow keras  linear modules etc.,. - high level primitives and build neural network - make sure you use the Model subclassing (not the functional api or sequential api) as shown in the slides of the class.

---



---






In [1]:
import numpy as np
import random
from numpy import array
from numpy.random import uniform
from numpy import hstack
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras import optimizers

In [2]:

n, d = 400, 3
#num_hidden1=5 # number of neurons for first hidden layer
#num_hidden2=4 # numbe rof neurons for second hidden layer
x = tf.random.uniform(minval=-1, maxval=1, shape = (n, d))
x2 = tf.cast(x**2, tf.float32)
weights_true = tf.cast(tf.constant([[5,1,1],[4,1,1]]),tf.float32)
bias_true = tf.cast(tf.constant([1,2]), tf.float32)
y_true = tf.matmul(x2, weights_true, transpose_b=True) + tf.matmul(x,weights_true,transpose_b=True) + bias_true

print(f'x: {x.shape}, weights: {weights_true.shape}, bias: {bias_true.shape}, y: {y_true.shape}')


x: (400, 3), weights: (2, 3), bias: (2,), y: (400, 2)


In [3]:
##Building 3 layer NN with Tensorflow using Model Subclassing API¶

class Linear(Layer):
  """y = w.x + b"""

  def __init__(self, units=32):
      super(Linear, self).__init__()
      self.units = units

  def build(self, input_shape):
      self.w = self.add_weight(shape=(input_shape[-1], self.units),
                               initializer='random_normal',
                               trainable=True)
      self.b = self.add_weight(shape=(self.units,),
                               initializer='random_normal',
                               trainable=True)

  def call(self, inputs):
      return tf.matmul(inputs, self.w) + self.b

In [4]:
##adding dropout

class Dropout(Layer):
  
  def __init__(self, rate):
    super(Dropout, self).__init__()
    self.rate = rate

  def call(self, inputs, training=None):
    if training:
      return tf.nn.dropout(inputs, rate=self.rate)
    return inputs

In [5]:
##MLP model class
class MLP(Layer):
    """Simple stack of Linear layers."""

    def __init__(self):
        super(MLP, self).__init__()
        self.linear_1 = Linear(22)
        self.dropout = Dropout(0.5)
        self.linear_2 = Linear(12)
        self.dropout = Dropout(0.5)
        self.linear_3 = Linear(2)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.dropout(x, training=None)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        x = self.dropout(x, training=None)
        return self.linear_3(x)

In [6]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001)
mse_loss_fn = tf.keras.losses.MeanSquaredError()
loss_metric = tf.keras.metrics.Mean()

dataset = tf.data.Dataset.from_tensor_slices((x, y_true))
dataset = dataset.shuffle(buffer_size=1).batch(30)

epochs = 2000
mlp = MLP()
# Iterate over epochs.
for epoch in range(epochs):

    # Iterate over the batches of the dataset.
    for step, (x_batch, y_batch) in enumerate(dataset):
        with tf.GradientTape() as tape:
            # Forward Pass
            y_pred_batch = mlp(x_batch)
            # Compute loss
            loss = mse_loss_fn(y_batch, y_pred_batch)
            loss += sum(mlp.losses)  # Add regularization loss

        grads = tape.gradient(loss, mlp.trainable_weights) # Use autograd
        optimizer.apply_gradients(zip(grads, mlp.trainable_weights)) # Update learnable parameter

        loss_metric(loss)
    if epoch % 100 == 0:
      print(f'Epoch {epoch}, loss {loss_metric.result()}')

Epoch 0, loss 25.410503387451172
Epoch 100, loss 23.409982681274414
Epoch 200, loss 21.71523094177246
Epoch 300, loss 20.23111343383789
Epoch 400, loss 18.88803482055664
Epoch 500, loss 17.664936065673828
Epoch 600, loss 16.594825744628906
Epoch 700, loss 15.707119941711426
Epoch 800, loss 14.978309631347656
Epoch 900, loss 14.348433494567871
Epoch 1000, loss 13.75150203704834
Epoch 1100, loss 13.13669204711914
Epoch 1200, loss 12.486892700195312
Epoch 1300, loss 11.816397666931152
Epoch 1400, loss 11.15234661102295
Epoch 1500, loss 10.519217491149902
Epoch 1600, loss 9.933116912841797
Epoch 1700, loss 9.401982307434082
Epoch 1800, loss 8.924427032470703
Epoch 1900, loss 8.49405288696289


In [7]:

def plot_intereactive_3d(x, y, y_pred=None):
  import plotly.graph_objects as go

  fig = go.Figure()
  fig.add_trace(go.Scatter3d(x = x[:,0],
                    y = x[:,1],
                    z = y.reshape([-1]),
                    opacity=0.5, mode='markers', name='Underlying Function'
                    ))
 
  if y_pred is not None:
    fig.add_trace(go.Scatter3d(x = x[:,0],
                   y = x[:,1],
                   z = y_pred.reshape([-1]),
                   opacity=0.5, mode='markers', name='Predicted Function'
                  ))
    
  fig.update_layout(scene = dict(
                    xaxis_title='X1',
                    yaxis_title='X2',
                    zaxis_title='Y'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))
  fig.show()

In [8]:
from sklearn.manifold import TSNE
X_red = TSNE(n_components=2).fit_transform(x)
y_true_red = TSNE(n_components=1).fit_transform(y_true)
y_pred_red = TSNE(n_components=1).fit_transform(mlp(x))
print(f'the X_red: {X_red.shape}, the y_true_red: {y_true_red.shape}, the y_pred_red: {y_pred_red.shape}')
plot_intereactive_3d(X_red,y_true_red,y_pred_red)

the X_red: (400, 2), the y_true_red: (400, 1), the y_pred_red: (400, 1)
